# Project1: Hyperdrive & AutoMl optimization and comparison

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-236086
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-236086


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code APXH3RMC3 to authenticate.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "project1ComputeClusterNK"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(
    vm_size = "STANDARD_D2_V2",
    min_nodes=0,
    max_nodes=4,
)

try:
    cluster_handle = ComputeTarget(workspace=ws, name=cluster_name)
    print("Get already existing cluster.")
except:
    print("Could not find such cluster. Create one.")
    cluster_handle = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
cluster_handle.wait_for_completion()


Could not find such cluster. Create one.
Provisioning operation finished, operation "Succeeded"


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(min_value=1e-2, max_value=5e2),
    "--max_iter": choice([25, 50, 75])
})
# Specify a Policy
policy = BanditPolicy(
    slack_factor=0.01,
    evaluation_interval=1, 
    delay_evaluation=5
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=cluster_handle,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=12,
)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run_with_hyperdrive_opt = exp.submit(config=hyperdrive_config)

In [7]:
RunDetails(run_with_hyperdrive_opt).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
best_result_experiment = run_with_hyperdrive_opt.get_best_run_by_primary_metric()

In [9]:
best_result_experiment.get_file_names()

['azureml-logs/20_image_build_log.txt',
 'logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-06-19-09',
 'outputs/.amlignore',
 'outputs/hyperdrive_best_model.pkl',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [10]:
import joblib
# Get your best run and save the model from that run.

best_result_experiment.download_file("outputs/model.pkl", "outputs/hyperdrive_best_model.pkl")
best_result_experiment.register_model(model_name="hyperdrive_opt_best_model", model_path=os.path.join("outputs", "model.pkl"))

Model(workspace=Workspace.create(name='quick-starts-ws-236086', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-236086'), name=hyperdrive_opt_best_model, id=hyperdrive_opt_best_model:1, version=1, tags={}, properties={})

## AutoMl optimization

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
DATA_URL =  "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=DATA_URL)

In [14]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [25]:
from azureml.core import Dataset

ds_cleaned = Dataset.Tabular.register_pandas_dataframe(x.join(y), target=ws.get_default_datastore(), name="bankmarketing-data")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/2c3353e9-d225-417c-b4e4-6f6b7fdd44ec/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds_cleaned,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cluster_handle,
    max_concurrent_iterations=4,
    enable_early_stopping=True,
    )

In [27]:
# Submit your automl run

run_with_autml_opt = exp.submit(config=automl_config, show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ceb3b5e7-a33b-4b76-9645-399c2979ab64,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
RunDetails(run_with_autml_opt).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [29]:
# Retrieve and save your best automl model.

best_run, best_model = run_with_autml_opt.get_output()
best_run.download_file(os.path.join("outputs", "model.pkl"), os.path.join("outputs", "automl_best_model.pkl"))
best_run.register_model(model_name="automl_opt_best_model", model_path=os.path.join("outputs", "model.pkl"))

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Model(workspace=Workspace.create(name='quick-starts-ws-236086', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55', resource_group='aml-quickstarts-236086'), name=automl_opt_best_model, id=automl_opt_best_model:1, version=1, tags={}, properties={})

In [30]:
# Delete compute cluster
cluster_handle.delete()